In [27]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.fftpack import fft
import plotly.express as px

import os

In [28]:
# Função de criação do espectro de FFT
def espectro_fft(x, Fs, pl = False):
    n = np.size(x)
    fr = (Fs/2)*np.linspace(0,1,int(n/2))
    # Compute FFT
    X = fft(x)
    X_m = (2/n)*abs(X[0:np.size(fr)])
    #print(X_m)
    #print(fr)
    if (pl == True):
        plt.plot(fr,X_m); plt.title('Magnitude Spectrum')
        title = r"Aceleração($m/s^2$)"
        plt.xlabel('Frequency(Hz)'); plt.ylabel(title)
        plt.tight_layout()
        plt.grid('on')
        plt.show()
    return fr, X_m

In [29]:
# https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
# The easiest way to upload a CSV file is from your GitHub repository. Click on the dataset in your repository, 
# then click on View Raw. Copy the link to the raw dataset and store it as a string variable called url in Colab 
# as shown below (a cleaner method but it’s not necessary). The last step is to load the url into Pandas read_csv to get the dataframe.

url_1 = 'https://raw.githubusercontent.com/gdnpleite/ifpe_vibmec/main/data/1_DOF_01-01-2000_00-04-58.csv'
url_0 = 'https://raw.githubusercontent.com/gdnpleite/ifpe_vibmec/main/data/1_DOF_01-01-2000_00-03-59.csv'

df = pd.read_csv(url_0) # leitura do dado do github

In [30]:
df.head()

,micros,ax,ay,az
0,0,17424,520,-2072
1,999,17376,476,-2076
2,2000,17356,524,-2036
3,3000,17412,516,-2064
4,3999,17432,520,-2152


In [31]:
df['seconds'] = df['micros']/1e6

In [32]:
df

,micros,ax,ay,az,seconds
0,0,17424,520,-2072,0.000000
1,999,17376,476,-2076,0.000999
2,2000,17356,524,-2036,0.002000
3,3000,17412,516,-2064,0.003000
4,3999,17432,520,-2152,0.003999
...,...,...,...,...,...
19995,19995000,17452,512,-1948,19.995000
19996,19996001,17424,544,-1952,19.996001
19997,19997000,17388,532,-2092,19.997000
19998,19998000,17408,500,-1952,19.998000


In [33]:
df = df.set_index('seconds')

In [34]:
df.drop('micros', axis = 1, inplace = True)

In [35]:
# Conversão para g
df = df * (9.81 / 16384.0)
df

,ax,ay,az
seconds,,,
0.000000,10.432705,0.311353,-1.240620
0.000999,10.403965,0.285007,-1.243015
0.002000,10.391990,0.313748,-1.219065
0.003000,10.425520,0.308958,-1.235830
0.003999,10.437495,0.311353,-1.288521
...,...,...,...
19.995000,10.449470,0.306563,-1.166375
19.996001,10.432705,0.325723,-1.168770
19.997000,10.411150,0.318538,-1.252595


In [36]:
df.ax = df.ax.values - np.mean(df.ax.values)
df.ay = df.ay.values - np.mean(df.ay.values)
df.az = df.az.values - np.mean(df.az.values)
df

,ax,ay,az
seconds,,,
0.000000,0.020316,0.023170,-0.014475
0.000999,-0.008424,-0.003175,-0.016870
0.002000,-0.020399,0.025565,0.007080
0.003000,0.013131,0.020775,-0.009685
0.003999,0.025106,0.023170,-0.062376
...,...,...,...
19.995000,0.037081,0.018380,0.059770
19.996001,0.020316,0.037540,0.057375
19.997000,-0.001239,0.030355,-0.026450


In [37]:
# calculando o dt
tam = df.shape[0]
dt_vec = (df.index[1:tam]-df.index[0:(tam-1)])
df_dt = pd.DataFrame(dt_vec)
#print(dt_vec[dt_vec>1])
fig = px.line(df_dt)
fig.show()

In [38]:
fs = 1/np.mean(dt_vec)
print(f"Frequência de amostragem = {fs} Hz")

Frequência de amostragem = 1000.0 Hz


In [39]:
f, A  = espectro_fft(df.ax.values, fs)

In [40]:
df_freq = pd.DataFrame(np.transpose([f, A]), columns=['Freq', 'Amp'])
df_freq

,Freq,Amp
0,0.000000,2.659384e-15
1,0.050005,6.874881e-04
2,0.100010,6.266260e-04
3,0.150015,5.602577e-04
4,0.200020,3.095551e-04
...,...,...
9995,499.799980,1.675175e-04
9996,499.849985,2.876212e-04
9997,499.899990,3.530651e-04
9998,499.949995,2.509195e-04


In [41]:
fig = px.line(df, x = df.index, y = 'ax')
fig.show()

In [42]:
fig = px.line(df_freq, x = 'Freq', y = 'Amp')
fig.show()